In [1]:
#Bag of words

In [17]:
#from pathlib import Path
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer
import re

In [18]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix

[nltk_data] Downloading package punkt to /home/kanervs4/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kanervs4/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
col_list = ["v1", "v2"]
sms_file = pd.read_csv('oikeesti.csv', encoding='latin-1', usecols=col_list)
sms_file = sms_file.rename(columns={"v1":"label", "v2":"text"})
type(sms_file)
sms_file.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [20]:
def clean_out(text):
    words = re.findall('[a-zA-Z0-9]+', text)
    words = word_tokenize(text)
    legit_words = [word for word in words if word not in stopwords.words("english")]
    
    
    text = " ".join(words)
    
    return text

In [21]:
sms_file["text"] = sms_file["text"].apply(str)

sms_file_X = [clean_out(text) for text in sms_file["text"]]

In [22]:
sms_file["label"].value_counts()

ham       4817
spam       746
ham"""       2
Name: label, dtype: int64

In [23]:
X_all = sms_file_X
y_all = sms_file["label"]
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.3, random_state=42)

In [24]:
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = None,
                             max_features = 100)

X_train_features = vectorizer.fit_transform(X_train)

X_train_features.shape

(3895, 100)

In [25]:
forest = RandomForestClassifier(n_estimators = 50)
forest = forest.fit(X_train_features, y_train)

In [26]:
X_test_features = vectorizer.transform(X_test)
y_pred = forest.predict(X_test_features)

testScore = f1_score(y_test, y_pred, pos_label='spam')

testScore

0.8930232558139535

In [27]:
cm = confusion_matrix(y_train[:1000], y_test[:1000])

In [28]:
test_sms = ["You have won FREE 1000€, get it right now! Gift for you winner! Please log in and call",
            "I love you, mom",
            "URGENT! Free concert tickets in your area, speak to a live operator 04033889 and get first time free",
           "call me back, we need to talk, its urgent."]

test_sms = vectorizer.transform(test_sms)
new_predict = forest.predict(test_sms)
print(new_predict)

['spam' 'ham' 'spam' 'ham']
